In [29]:
import pandas as pd
import json
from pprint import pprint

In [24]:
# load festivals data
festivals_raw = json.load(open('all_festival_bands_2017.json'))

In [42]:
# extract genres (and other info) of bands in festivals
festivals_list = []
for f in festivals_raw:
    for b in f['bands']:
        #print('{} - {} - {} - {} - {}'.format(f['name'], b['name'], b['popularity'], b['followers']['total'], b['genres']))
        festivals_list.append({'festival': f['name'], 
                               'band' : b['name'], 
                               'popularity': b['popularity'], 
                               'followers' : b['followers']['total'], 
                               'genres' : b['genres']})

# convert to pandas DF
festivals = pd.DataFrame.from_dict(festivals)
festivals.head()

,band,festival,followers,genres,popularity
0,SFDK,al_rumbo,151813,"[deep latin hip hop, latin hip hop, spanish hi...",54
1,Mala Rodríguez,al_rumbo,100795,"[deep latin hip hop, flamenco, latin alternati...",51
2,Estricnina,al_rumbo,6809,[spanish rock],38
3,La Raíz,al_rumbo,73371,"[spanish punk, spanish rock]",54
4,La Pegatina,al_rumbo,105792,"[spanish pop, spanish punk, spanish rock]",51


## Explore genres

In [51]:
all_genres = set(festivals['genres'].sum())

In [52]:
all_genres

{'acid house',
 'acid jazz',
 'acid techno',
 'african rock',
 'afrobeat',
 'album rock',
 'alt-indie rock',
 'alternative country',
 'alternative dance',
 'alternative emo',
 'alternative hip hop',
 'alternative metal',
 'alternative pop',
 'alternative pop rock',
 'alternative rock',
 'ambient',
 'anthem emo',
 'anti-folk',
 'argentine rock',
 'art rock',
 'athens indie',
 'atmospheric black metal',
 'aussietronica',
 'austindie',
 'australian alternative rock',
 'australian dance',
 'avantgarde metal',
 'bachata',
 'balearic',
 'basque rock',
 'bass music',
 'bass trap',
 'beatdown',
 'belgian rock',
 'big beat',
 'big room',
 'black metal',
 'black sludge',
 'black thrash',
 'blackgaze',
 'blues',
 'blues-rock',
 'bossa nova',
 'boston rock',
 'bow pop',
 'brazilian indie',
 'brazilian punk',
 'breakbeat',
 'brill building pop',
 'british alternative rock',
 'british blues',
 'britpop',
 'brooklyn indie',
 'brostep',
 'brutal death metal',
 'c86',
 'cabaret',
 'canadian indie',
 'c